# 🔍 DETECCIÓN AUTOMÁTICA DE ANOMALÍAS - AGOSTO 2025
## Sistema de Inferencia Real para Mantenimiento Predictivo

**Autor:** Antonio Cantos & Renzo Chavez - TFM EADIC - 2025

### 🎯 OBJETIVO ESPECÍFICO:
Demostrar la detección automática de anomalías en datos reales de agosto 2025 usando el modelo entrenado, sin conocimiento previo de qué buscar.

### 🧠 PROCESO DE INFERENCIA AUTOMÁTICA:
1. **Cargar modelo entrenado** → Sin saber qué anomalías buscar
2. **Procesar datos agosto** → Aplicar misma limpieza que entrenamiento
3. **Aplicar algoritmo ML** → Isolation Forest decide qué es anómalo
4. **Clasificar severidad** → Basado en THD y scores
5. **Generar diagnósticos** → Automáticos según patrones
6. **Crear alertas** → Priorizadas por criticidad

### ⚡ VENTAJAS DEL SISTEMA:
- **Detección 24/7** sin intervención humana
- **Análisis objetivo** basado en patrones aprendidos
- **Clasificación automática** por severidad
- **Diagnósticos específicos** para cada anomalía
- **Trazabilidad completa** de decisiones del modelo

In [ ]:
# 1. 🔧 CONFIGURACIÓN INICIAL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import os
from datetime import datetime, timedelta
import json
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

# Rutas de trabajo
BASE_PATH = Path('.')
OUTPUT_PATH = BASE_PATH / 'output'
MODELS_PATH = OUTPUT_PATH / 'models'

# Crear directorios
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)

print("🔍 SISTEMA DE DETECCIÓN DE ANOMALÍAS INICIADO")
print(f"📁 Directorio de trabajo: {BASE_PATH}")
print(f"📁 Salida: {OUTPUT_PATH}")
print("=" * 50)

In [ ]:
# 2. 📥 CARGAR MODELO ENTRENADO
print("📥 CARGANDO MODELO ENTRENADO")
print("=" * 30)

# Buscar modelo en diferentes ubicaciones
posibles_rutas_modelo = [
    MODELS_PATH / 'modelo_predictivo_tfm.pkl',
    OUTPUT_PATH / 'modelo_predictivo_tfm.pkl',
    BASE_PATH / 'modelo_predictivo_tfm.pkl'
]

modelo_cargado = None
scaler = None
variables_modelo = None
parametros_modelo = None

for ruta in posibles_rutas_modelo:
    if ruta.exists():
        try:
            print(f"🔄 Intentando cargar desde: {ruta}")
            datos_modelo = joblib.load(ruta)
            
            if isinstance(datos_modelo, dict):
                modelo_cargado = datos_modelo.get('modelo')
                scaler = datos_modelo.get('scaler')
                variables_modelo = datos_modelo.get('variables', [])
                parametros_modelo = datos_modelo.get('parametros', {})
            else:
                modelo_cargado = datos_modelo
                variables_modelo = []
                parametros_modelo = {}
            
            print(f"✅ Modelo cargado exitosamente desde: {ruta}")
            break
            
        except Exception as e:
            print(f"❌ Error cargando desde {ruta}: {e}")
            continue

if modelo_cargado is None:
    print("\n❌ NO SE ENCONTRÓ MODELO ENTRENADO")
    print("💡 Ejecuta primero el notebook completo para entrenar el modelo")
    print("📋 O asegúrate de tener el archivo modelo_predictivo_tfm.pkl")
    raise FileNotFoundError("Modelo entrenado requerido")

# Mostrar información del modelo
print(f"\n🧠 INFORMACIÓN DEL MODELO:")
print(f"   Tipo: {type(modelo_cargado).__name__}")
if hasattr(modelo_cargado, 'contamination'):
    print(f"   Contaminación: {modelo_cargado.contamination}")
if hasattr(modelo_cargado, 'n_estimators'):
    print(f"   Estimadores: {modelo_cargado.n_estimators}")

if variables_modelo:
    print(f"\n📊 VARIABLES DEL MODELO: {len(variables_modelo)}")
    for i, var in enumerate(variables_modelo, 1):
        print(f"   {i:2d}. {var}")

if parametros_modelo:
    print(f"\n⚙️ PARÁMETROS DEL MODELO:")
    for param, valor in parametros_modelo.items():
        print(f"   {param}: {valor}")

# Parámetros de referencia para análisis
umbrales_thd = {
    'critico': 4.0,
    'alerta': 2.0,
    'atencion': 1.5,
    'normal': 1.0
}

print(f"\n🎯 UMBRALES THD CONFIGURADOS:")
for nivel, umbral in umbrales_thd.items():
    print(f"   {nivel.upper()}: > {umbral}")

print("\n✅ MODELO LISTO PARA DETECCIÓN")

In [ ]:
# 3. 📊 CARGAR Y PROCESAR DATOS DE AGOSTO
print("📊 CARGANDO DATOS DE AGOSTO 2025")
print("=" * 35)
print("⚠️ PROCESANDO SIN CONOCIMIENTO PREVIO DE ANOMALÍAS")

# Función de limpieza (misma que entrenamiento)
def limpiar_datos_agosto(archivo_path):
    """Limpia datos de agosto usando el mismo proceso que entrenamiento"""
    
    print(f"\n🧹 Procesando: {archivo_path}")
    
    try:
        # Cargar datos
        df = pd.read_excel(archivo_path)
        print(f" 📊 Datos originales: {len(df):,} registros, {len(df.columns)} columnas")
        
        # Mapeo de columnas (mismo que entrenamiento)
        mapeo_columnas = {}
        
        for col in df.columns:
            col_str = str(col)
            
            # Timestamp
            if 'Hora' in col_str or 'Time' in col_str:
                mapeo_columnas[col] = 'timestamp'
            
            # THD Variables
            elif 'Distorsión Armónica Total' in col_str:
                if 'Actual' in col_str and 'Fase A' in col_str:
                    mapeo_columnas[col] = 'THD_Corriente_A'
                elif 'Actual' in col_str and 'Fase B' in col_str:
                    mapeo_columnas[col] = 'THD_Corriente_B'
                elif 'Actual' in col_str and 'Fase C' in col_str:
                    mapeo_columnas[col] = 'THD_Corriente_C'
                elif 'Voltaje' in col_str and 'Fase A' in col_str:
                    mapeo_columnas[col] = 'THD_Voltaje_A'
                elif 'Voltaje' in col_str and 'Fase B' in col_str:
                    mapeo_columnas[col] = 'THD_Voltaje_B'
                elif 'Voltaje' in col_str and 'Fase C' in col_str:
                    mapeo_columnas[col] = 'THD_Voltaje_C'
                elif 'Voltaje' in col_str and 'Promedio' in col_str:
                    mapeo_columnas[col] = 'THD_Voltaje_Promedio'
            
            # Potencia
            elif 'Demanda' in col_str and 'kW' in col_str:
                if 'Fase A' in col_str:
                    mapeo_columnas[col] = 'Potencia_A'
                elif 'Fase B' in col_str:
                    mapeo_columnas[col] = 'Potencia_B'
                elif 'Fase C' in col_str:
                    mapeo_columnas[col] = 'Potencia_C'
                elif 'Fase' not in col_str:
                    mapeo_columnas[col] = 'Potencia_Total'
        
        # Aplicar mapeo
        if mapeo_columnas:
            df = df.rename(columns=mapeo_columnas)
            print(f" ✅ Columnas mapeadas: {len(mapeo_columnas)}")
        
        # Procesar timestamp
        if 'timestamp' in df.columns:
            df['timestamp'] = df['timestamp'].astype(str)
            df['timestamp'] = df['timestamp'].str.replace(' GMT-3', '', regex=False)
            df['timestamp'] = df['timestamp'].str.replace(' GMT-4', '', regex=False)
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
            df = df.dropna(subset=['timestamp'])
            df = df.sort_values('timestamp')
            print(f" ✅ Timestamp procesado: {len(df):,} registros válidos")
        
        # Convertir a numérico
        for col in df.columns:
            if col not in ['timestamp']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calcular THD agregado
        columnas_thd = [col for col in df.columns if 'THD' in col and col != 'THD']
        if columnas_thd:
            df['THD'] = df[columnas_thd].mean(axis=1)
            print(f" ✅ THD calculado: {len(columnas_thd)} variables → THD agregado")
        
        # Estados operacionales
        columnas_potencia = [col for col in df.columns if 'Potencia' in col]
        if columnas_potencia:
            potencia_total = df[columnas_potencia].sum(axis=1)
            df['Estado'] = 'FUNCIONANDO'
            df.loc[potencia_total <= 1.0, 'Estado'] = 'PARADO'
            
            funcionando = (df['Estado'] == 'FUNCIONANDO').sum()
            parado = (df['Estado'] == 'PARADO').sum()
            
            print(f" ✅ Estados: FUNCIONANDO {funcionando:,} ({funcionando/len(df)*100:.1f}%), PARADO {parado:,} ({parado/len(df)*100:.1f}%)")
        
        print(f" 🎉 Procesamiento exitoso: {len(df):,} registros finales")
        return df
        
    except Exception as e:
        print(f" ❌ Error: {e}")
        return None

# Cargar datos de agosto
archivo_agosto = BASE_PATH / 'InformacionAgosto_fp1.xlsx'

if not archivo_agosto.exists():
    print(f"❌ Archivo no encontrado: {archivo_agosto}")
    print("💡 Coloca el archivo InformacionAgosto_fp1.xlsx en el directorio actual")
    raise FileNotFoundError("Archivo de datos de agosto requerido")

# Procesar datos
datos_agosto = limpiar_datos_agosto(archivo_agosto)

if datos_agosto is None:
    raise ValueError("Error procesando datos de agosto")

# Resumen de datos procesados
print(f"\n📊 RESUMEN DATOS AGOSTO PROCESADOS:")
print(f"   Total registros: {len(datos_agosto):,}")
print(f"   Período: {datos_agosto['timestamp'].min()} a {datos_agosto['timestamp'].max()}")
print(f"   Días de datos: {(datos_agosto['timestamp'].max() - datos_agosto['timestamp'].min()).days + 1} días")

# Estadísticas THD preliminares
if 'THD' in datos_agosto.columns:
    thd_stats = datos_agosto['THD'].describe()
    print(f"\n📈 ESTADÍSTICAS THD (SIN ANÁLISIS):")
    print(f"   Registros válidos: {datos_agosto['THD'].notna().sum():,}")
    print(f"   Promedio: {thd_stats['mean']:.3f}")
    print(f"   Mediana: {thd_stats['50%']:.3f}")
    print(f"   Mínimo: {thd_stats['min']:.3f}")
    print(f"   Máximo: {thd_stats['max']:.3f}")
    print(f"   Desviación: {thd_stats['std']:.3f}")

print("\n✅ DATOS AGOSTO LISTOS PARA DETECCIÓN DE ANOMALÍAS")

In [ ]:
# 4. 🧠 APLICAR MODELO DE DETECCIÓN DE ANOMALÍAS
print("🧠 APLICANDO MODELO DE DETECCIÓN DE ANOMALÍAS")
print("=" * 50)
print("⚡ EL MODELO ISOLATION FOREST DECIDE QUÉ ES ANÓMALO")
print("🔍 SIN CONOCIMIENTO PREVIO - DETECCIÓN AUTOMÁTICA")

# Preparar variables para el modelo
if variables_modelo:
    # Usar variables del modelo entrenado
    variables_disponibles = [var for var in variables_modelo if var in datos_agosto.columns]
    print(f"\n📊 Variables del modelo entrenado: {len(variables_modelo)}")
    print(f"📊 Variables disponibles en agosto: {len(variables_disponibles)}")
else:
    # Variables por defecto
    variables_disponibles = []
    for var in ['THD_Corriente_A', 'THD_Corriente_B', 'THD_Corriente_C', 
                'THD_Voltaje_A', 'THD_Voltaje_B', 'THD_Voltaje_C', 'THD_Voltaje_Promedio', 'THD']:
        if var in datos_agosto.columns:
            variables_disponibles.append(var)
    
    for var in ['Potencia_A', 'Potencia_B', 'Potencia_C', 'Potencia_Total']:
        if var in datos_agosto.columns:
            variables_disponibles.append(var)

print(f"\n🔍 VARIABLES PARA DETECCIÓN: {len(variables_disponibles)}")
for i, var in enumerate(variables_disponibles, 1):
    disponible = "✅" if var in datos_agosto.columns else "❌"
    print(f"   {i:2d}. {var} {disponible}")

if len(variables_disponibles) == 0:
    raise ValueError("No hay variables disponibles para detección")

# Preparar datos para el modelo
print(f"\n🔄 PREPARANDO DATOS PARA INFERENCIA...")

# Filtrar solo datos en funcionamiento (como en entrenamiento)
if 'Estado' in datos_agosto.columns:
    datos_funcionando = datos_agosto[datos_agosto['Estado'] == 'FUNCIONANDO'].copy()
    print(f"   Filtrando FUNCIONAMIENTO: {len(datos_funcionando):,} registros")
else:
    datos_funcionando = datos_agosto.copy()
    print(f"   Sin filtro de estado: {len(datos_funcionando):,} registros")

# Extraer variables para el modelo
X_agosto = datos_funcionando[variables_disponibles].copy()

# Eliminar registros con valores nulos
X_agosto_limpio = X_agosto.dropna()
indices_validos = X_agosto_limpio.index

print(f"   Después de limpiar nulos: {len(X_agosto_limpio):,} registros")

if len(X_agosto_limpio) == 0:
    raise ValueError("No hay datos válidos para detección")

# Normalizar datos
print(f"\n🔄 NORMALIZANDO DATOS...")

if scaler is not None:
    # Usar scaler del entrenamiento
    try:
        X_agosto_normalizado = scaler.transform(X_agosto_limpio)
        print(f"   ✅ Usando scaler del entrenamiento")
    except Exception as e:
        print(f"   ⚠️ Error con scaler entrenado: {e}")
        print(f"   🔄 Creando scaler temporal...")
        scaler_temp = StandardScaler()
        X_agosto_normalizado = scaler_temp.fit_transform(X_agosto_limpio)
else:
    # Crear scaler temporal
    print(f"   🔄 Creando scaler temporal...")
    scaler_temp = StandardScaler()
    X_agosto_normalizado = scaler_temp.fit_transform(X_agosto_limpio)

print(f"   ✅ Datos normalizados: {X_agosto_normalizado.shape}")

# APLICAR MODELO DE DETECCIÓN
print(f"\n🧠 APLICANDO ISOLATION FOREST...")
print(f"   Registros a analizar: {len(X_agosto_normalizado):,}")
print(f"   Variables: {len(variables_disponibles)}")

# Obtener scores y predicciones
scores_anomalia = modelo_cargado.decision_function(X_agosto_normalizado)
predicciones_anomalia = modelo_cargado.predict(X_agosto_normalizado)

# Crear DataFrame con resultados
resultados_deteccion = datos_funcionando.loc[indices_validos].copy()
resultados_deteccion['score_anomalia'] = scores_anomalia
resultados_deteccion['es_anomalia'] = predicciones_anomalia == -1

# ANÁLISIS DE RESULTADOS
total_registros = len(predicciones_anomalia)
total_anomalias = (predicciones_anomalia == -1).sum()
porcentaje_anomalias = total_anomalias / total_registros * 100

print(f"\n📊 RESULTADOS DE DETECCIÓN AUTOMÁTICA:")
print(f"   Registros analizados: {total_registros:,}")
print(f"   Anomalías detectadas: {total_anomalias:,} ({porcentaje_anomalias:.2f}%)")
print(f"   Registros normales: {total_registros - total_anomalias:,} ({100-porcentaje_anomalias:.2f}%)")

print(f"\n📈 ESTADÍSTICAS DE SCORES:")
print(f"   Score promedio: {scores_anomalia.mean():.3f}")
print(f"   Score mínimo (más anómalo): {scores_anomalia.min():.3f}")
print(f"   Score máximo (más normal): {scores_anomalia.max():.3f}")
print(f"   Desviación estándar: {scores_anomalia.std():.3f}")

if total_anomalias > 0:
    scores_anomalias = scores_anomalia[predicciones_anomalia == -1]
    print(f"\n🚨 ESTADÍSTICAS DE ANOMALÍAS:")
    print(f"   Score promedio anomalías: {scores_anomalias.mean():.3f}")
    print(f"   Score más crítico: {scores_anomalias.min():.3f}")
    print(f"   Score menos crítico: {scores_anomalias.max():.3f}")

print("\n✅ DETECCIÓN DE ANOMALÍAS COMPLETADA")

In [ ]:
# 5. 🏷️ CLASIFICACIÓN AUTOMÁTICA POR SEVERIDAD
print("🏷️ CLASIFICACIÓN AUTOMÁTICA POR SEVERIDAD")
print("=" * 45)

if total_anomalias > 0:
    # Extraer anomalías detectadas
    anomalias_detectadas = resultados_deteccion[resultados_deteccion['es_anomalia']].copy()
    
    print(f"🔍 Clasificando {len(anomalias_detectadas)} anomalías detectadas...")
    
    # Función de clasificación por severidad
    def clasificar_severidad_automatica(row):
        """Clasifica automáticamente la severidad basada en THD y score"""
        thd = row.get('THD', 0)
        score = row['score_anomalia']
        
        # Criterios de clasificación automática
        if thd > umbrales_thd['critico'] or score < -0.5:
            return 'CRÍTICO'
        elif thd > umbrales_thd['alerta'] or score < -0.3:
            return 'ALERTA'
        elif thd > umbrales_thd['atencion'] or score < -0.1:
            return 'ATENCIÓN'
        else:
            return 'MENOR'
    
    # Aplicar clasificación
    anomalias_detectadas['severidad'] = anomalias_detectadas.apply(clasificar_severidad_automatica, axis=1)
    
    # Función de diagnóstico automático
    def generar_diagnostico_automatico(row):
        """Genera diagnóstico automático basado en parámetros"""
        thd = row.get('THD', 0)
        score = row['score_anomalia']
        severidad = row['severidad']
        timestamp = row['timestamp']
        
        # Diagnósticos específicos por severidad
        diagnosticos = {
            'CRÍTICO': f"ANOMALÍA CRÍTICA - THD: {thd:.3f} excede umbral crítico ({umbrales_thd['critico']}) - Score: {score:.3f} - Requiere intervención inmediata",
            'ALERTA': f"ANOMALÍA SIGNIFICATIVA - THD: {thd:.3f} en nivel de alerta - Score: {score:.3f} - Monitoreo intensivo requerido",
            'ATENCIÓN': f"ANOMALÍA MODERADA - THD: {thd:.3f} requiere atención - Score: {score:.3f} - Incluir en próximo mantenimiento",
            'MENOR': f"ANOMALÍA MENOR - THD: {thd:.3f} - Score: {score:.3f} - Monitoreo rutinario"
        }
        
        return diagnosticos.get(severidad, f"Anomalía detectada - THD: {thd:.3f} - Score: {score:.3f}")
    
    # Generar diagnósticos automáticos
    anomalias_detectadas['diagnostico'] = anomalias_detectadas.apply(generar_diagnostico_automatico, axis=1)
    
    # Agregar información adicional
    anomalias_detectadas['fecha_deteccion'] = datetime.now()
    anomalias_detectadas['metodo_deteccion'] = 'Isolation Forest'
    anomalias_detectadas['confianza'] = np.abs(anomalias_detectadas['score_anomalia'])  # Mayor valor absoluto = mayor confianza
    
    # ESTADÍSTICAS POR SEVERIDAD
    severidad_counts = anomalias_detectadas['severidad'].value_counts()
    
    print(f"\n🚨 DISTRIBUCIÓN POR SEVERIDAD:")
    colores_severidad = {'CRÍTICO': '🔴', 'ALERTA': '🟠', 'ATENCIÓN': '🟡', 'MENOR': '🔵'}
    
    for severidad in ['CRÍTICO', 'ALERTA', 'ATENCIÓN', 'MENOR']:
        cantidad = severidad_counts.get(severidad, 0)
        porcentaje = cantidad / len(anomalias_detectadas) * 100 if len(anomalias_detectadas) > 0 else 0
        color = colores_severidad.get(severidad, '⚪')
        print(f"   {color} {severidad}: {cantidad:,} anomalías ({porcentaje:.1f}%)")
    
    # TOP ANOMALÍAS MÁS CRÍTICAS
    print(f"\n🔍 TOP 10 ANOMALÍAS MÁS CRÍTICAS:")
    print("=" * 50)
    
    # Ordenar por severidad y score
    orden_severidad = {'CRÍTICO': 1, 'ALERTA': 2, 'ATENCIÓN': 3, 'MENOR': 4}
    anomalias_detectadas['orden_severidad'] = anomalias_detectadas['severidad'].map(orden_severidad)
    top_anomalias = anomalias_detectadas.nsmallest(10, ['orden_severidad', 'score_anomalia'])
    
    for i, (idx, anomalia) in enumerate(top_anomalias.iterrows(), 1):
        fecha = anomalia['timestamp'].strftime('%Y-%m-%d %H:%M')
        severidad = anomalia['severidad']
        thd = anomalia.get('THD', 0)
        score = anomalia['score_anomalia']
        color = colores_severidad.get(severidad, '⚪')
        
        print(f"   {i:2d}. {color} {fecha} - {severidad}")
        print(f"       THD: {thd:.3f} | Score: {score:.3f} | Confianza: {anomalia['confianza']:.3f}")
        print(f"       Diagnóstico: {anomalia['diagnostico'][:80]}...")
        print()
    
    # ANÁLISIS TEMPORAL
    print(f"\n⏰ ANÁLISIS TEMPORAL DE ANOMALÍAS:")
    
    # Anomalías por día
    anomalias_detectadas['fecha'] = anomalias_detectadas['timestamp'].dt.date
    anomalias_por_dia = anomalias_detectadas['fecha'].value_counts().sort_index()
    
    print(f"   Días con anomalías: {len(anomalias_por_dia)} de {(datos_agosto['timestamp'].max().date() - datos_agosto['timestamp'].min().date()).days + 1} días totales")
    print(f"   Día con más anomalías: {anomalias_por_dia.index[0]} ({anomalias_por_dia.iloc[0]} anomalías)")
    print(f"   Promedio anomalías/día: {anomalias_por_dia.mean():.1f}")
    
    # Anomalías por hora del día
    anomalias_detectadas['hora'] = anomalias_detectadas['timestamp'].dt.hour
    anomalias_por_hora = anomalias_detectadas['hora'].value_counts().sort_index()
    
    if len(anomalias_por_hora) > 0:
        hora_max = anomalias_por_hora.index[0]
        print(f"   Hora con más anomalías: {hora_max:02d}:00 ({anomalias_por_hora.iloc[0]} anomalías)")
    
    # Guardar resultados detallados
    archivo_anomalias = OUTPUT_PATH / 'anomalias_detectadas_agosto.csv'
    anomalias_detectadas.to_csv(archivo_anomalias, index=False)
    print(f"\n💾 Anomalías detalladas guardadas en: {archivo_anomalias}")
    
else:
    print("✅ NO SE DETECTARON ANOMALÍAS EN AGOSTO 2025")
    print("   El sistema indica operación normal durante todo el período")
    print("   Todos los parámetros dentro de rangos esperados")
    anomalias_detectadas = pd.DataFrame()

print("\n✅ CLASIFICACIÓN POR SEVERIDAD COMPLETADA")

In [ ]:
# 6. 📊 VISUALIZACIÓN DE RESULTADOS DE DETECCIÓN
print("📊 GENERANDO VISUALIZACIONES DE DETECCIÓN")
print("=" * 45)

# Configurar estilo
plt.style.use('default')
fig_size = (16, 12)

if len(resultados_deteccion) > 0:
    
    # Crear figura principal con subplots
    fig, axes = plt.subplots(4, 1, figsize=fig_size)
    fig.suptitle('🔍 Detección Automática de Anomalías - Agosto 2025\nSistema Isolation Forest', 
                 fontsize=16, fontweight='bold')
    
    # 1. THD con anomalías marcadas
    if 'THD' in resultados_deteccion.columns:
        # Línea base de THD
        axes[0].plot(resultados_deteccion['timestamp'], resultados_deteccion['THD'], 
                    color='blue', alpha=0.6, linewidth=1, label='THD Normal')
        
        # Marcar anomalías por severidad
        if len(anomalias_detectadas) > 0:
            colores_plot = {'CRÍTICO': 'red', 'ALERTA': 'orange', 'ATENCIÓN': 'yellow', 'MENOR': 'lightblue'}
            
            for severidad, color in colores_plot.items():
                anomalias_sev = anomalias_detectadas[anomalias_detectadas['severidad'] == severidad]
                if len(anomalias_sev) > 0:
                    axes[0].scatter(anomalias_sev['timestamp'], anomalias_sev['THD'], 
                                  color=color, s=60, alpha=0.8, label=f'Anomalías {severidad}', 
                                  zorder=5, edgecolors='black', linewidth=0.5)
        
        # Líneas de umbral
        axes[0].axhline(y=umbrales_thd['critico'], color='red', linestyle='--', alpha=0.7, label='Umbral Crítico')
        axes[0].axhline(y=umbrales_thd['alerta'], color='orange', linestyle='--', alpha=0.7, label='Umbral Alerta')
        axes[0].axhline(y=umbrales_thd['atencion'], color='gold', linestyle='--', alpha=0.7, label='Umbral Atención')
        
        axes[0].set_title('THD (Total Harmonic Distortion) con Anomalías Detectadas')
        axes[0].set_ylabel('THD')
        axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(True, alpha=0.3)
    
    # 2. Scores de anomalía
    axes[1].plot(resultados_deteccion['timestamp'], resultados_deteccion['score_anomalia'], 
                color='green', alpha=0.6, linewidth=1, label='Score Anomalía')
    
    if len(anomalias_detectadas) > 0:
        for severidad, color in colores_plot.items():
            anomalias_sev = anomalias_detectadas[anomalias_detectadas['severidad'] == severidad]
            if len(anomalias_sev) > 0:
                axes[1].scatter(anomalias_sev['timestamp'], anomalias_sev['score_anomalia'], 
                              color=color, s=60, alpha=0.8, label=f'{severidad}', 
                              zorder=5, edgecolors='black', linewidth=0.5)
    
    axes[1].axhline(y=-0.1, color='red', linestyle='--', alpha=0.7, label='Umbral Detección')
    axes[1].set_title('Scores de Detección de Anomalías (Isolation Forest)')
    axes[1].set_ylabel('Score')
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    # 3. Estados operacionales con anomalías
    if 'Estado' in resultados_deteccion.columns:
        estados_num = resultados_deteccion['Estado'].map({'PARADO': 0, 'FUNCIONANDO': 1})
        axes[2].fill_between(resultados_deteccion['timestamp'], estados_num, 
                           alpha=0.4, color='lightblue', label='Estado Operacional')
        
        if len(anomalias_detectadas) > 0:
            anomalias_estados = anomalias_detectadas['Estado'].map({'PARADO': 0, 'FUNCIONANDO': 1})
            axes[2].scatter(anomalias_detectadas['timestamp'], anomalias_estados, 
                          color='red', s=50, alpha=0.8, label='Anomalías Detectadas', zorder=5)
        
        axes[2].set_title('Estados Operacionales con Anomalías')
        axes[2].set_ylabel('Estado')
        axes[2].set_yticks([0, 1])
        axes[2].set_yticklabels(['PARADO', 'FUNCIONANDO'])
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
    
    # 4. Distribución temporal de anomalías
    if len(anomalias_detectadas) > 0:
        # Crear bins por día
        fechas_unicas = sorted(anomalias_detectadas['fecha'].unique())
        anomalias_por_dia_plot = anomalias_detectadas['fecha'].value_counts().sort_index()
        
        axes[3].bar(anomalias_por_dia_plot.index, anomalias_por_dia_plot.values, 
                   alpha=0.7, color='red', edgecolor='black', linewidth=0.5)
        axes[3].set_title('Distribución Temporal de Anomalías por Día')
        axes[3].set_ylabel('Cantidad de Anomalías')
        axes[3].tick_params(axis='x', rotation=45)
        axes[3].grid(True, alpha=0.3)
    else:
        axes[3].text(0.5, 0.5, 'No se detectaron anomalías\nen el período analizado', 
                    ha='center', va='center', transform=axes[3].transAxes, 
                    fontsize=14, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
        axes[3].set_title('Distribución Temporal de Anomalías')
    
    axes[3].set_xlabel('Fecha')
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar gráfico principal
    archivo_grafico_principal = OUTPUT_PATH / 'deteccion_anomalias_agosto.png'
    plt.savefig(archivo_grafico_principal, dpi=300, bbox_inches='tight')
    print(f"📊 Gráfico principal guardado: {archivo_grafico_principal}")
    plt.show()
    
    # Gráfico adicional: Análisis detallado de anomalías
    if len(anomalias_detectadas) > 0:
        fig2, axes2 = plt.subplots(2, 2, figsize=(15, 10))
        fig2.suptitle('🔍 Análisis Detallado de Anomalías Detectadas', fontsize=16, fontweight='bold')
        
        # Distribución por severidad
        severidad_counts = anomalias_detectadas['severidad'].value_counts()
        colores_barras = [colores_plot.get(sev, 'gray') for sev in severidad_counts.index]
        
        bars = axes2[0,0].bar(severidad_counts.index, severidad_counts.values, color=colores_barras)
        axes2[0,0].set_title('Distribución por Severidad')
        axes2[0,0].set_ylabel('Cantidad')
        
        # Agregar valores en las barras
        for bar in bars:
            height = bar.get_height()
            axes2[0,0].text(bar.get_x() + bar.get_width()/2., height,
                           f'{int(height)}', ha='center', va='bottom')
        
        # Histograma de THD en anomalías
        if 'THD' in anomalias_detectadas.columns:
            axes2[0,1].hist(anomalias_detectadas['THD'], bins=15, alpha=0.7, color='red', edgecolor='black')
            axes2[0,1].axvline(umbrales_thd['critico'], color='red', linestyle='--', label='Umbral Crítico')
            axes2[0,1].axvline(umbrales_thd['alerta'], color='orange', linestyle='--', label='Umbral Alerta')
            axes2[0,1].set_title('Distribución THD en Anomalías')
            axes2[0,1].set_xlabel('THD')
            axes2[0,1].set_ylabel('Frecuencia')
            axes2[0,1].legend()
        
        # Histograma de scores
        axes2[1,0].hist(anomalias_detectadas['score_anomalia'], bins=15, alpha=0.7, color='green', edgecolor='black')
        axes2[1,0].set_title('Distribución Scores de Anomalía')
        axes2[1,0].set_xlabel('Score')
        axes2[1,0].set_ylabel('Frecuencia')
        
        # Anomalías por hora del día
        anomalias_por_hora_plot = anomalias_detectadas['hora'].value_counts().sort_index()
        axes2[1,1].bar(anomalias_por_hora_plot.index, anomalias_por_hora_plot.values, 
                      alpha=0.7, color='purple', edgecolor='black')
        axes2[1,1].set_title('Anomalías por Hora del Día')
        axes2[1,1].set_xlabel('Hora')
        axes2[1,1].set_ylabel('Cantidad')
        axes2[1,1].set_xticks(range(0, 24, 2))
        
        plt.tight_layout()
        
        # Guardar gráfico de análisis
        archivo_grafico_analisis = OUTPUT_PATH / 'analisis_detallado_anomalias.png'
        plt.savefig(archivo_grafico_analisis, dpi=300, bbox_inches='tight')
        print(f"📊 Gráfico de análisis guardado: {archivo_grafico_analisis}")
        plt.show()

else:
    print("⚠️ No hay datos de detección para visualizar")

# Guardar todos los resultados
archivo_resultados_completos = OUTPUT_PATH / 'resultados_deteccion_completos.csv'
resultados_deteccion.to_csv(archivo_resultados_completos, index=False)
print(f"\n💾 Resultados completos guardados: {archivo_resultados_completos}")

print("\n✅ VISUALIZACIONES DE DETECCIÓN COMPLETADAS")

In [ ]:
# 7. 📋 RESUMEN EJECUTIVO DE DETECCIÓN
print("📋 GENERANDO RESUMEN EJECUTIVO DE DETECCIÓN")
print("=" * 50)

# Generar resumen completo
resumen_deteccion = f"""
# 🔍 RESUMEN EJECUTIVO - DETECCIÓN AUTOMÁTICA DE ANOMALÍAS
## Sistema de Mantenimiento Predictivo - Agosto 2025

**Fecha de Análisis:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Período Analizado:** {datos_agosto['timestamp'].min()} a {datos_agosto['timestamp'].max()}
**Método:** Isolation Forest + Análisis THD
**Estado:** Detección automática sin conocimiento previo

---

## 📊 RESULTADOS DE DETECCIÓN

### Datos Procesados:
- **Total registros agosto:** {len(datos_agosto):,}
- **Registros analizados:** {len(resultados_deteccion):,}
- **Variables monitoreadas:** {len(variables_disponibles)}
- **Días de datos:** {(datos_agosto['timestamp'].max().date() - datos_agosto['timestamp'].min().date()).days + 1} días

### Anomalías Detectadas:
- **Total anomalías:** {total_anomalias:,}
- **Porcentaje:** {porcentaje_anomalias:.2f}% del total
- **Registros normales:** {total_registros - total_anomalias:,} ({100-porcentaje_anomalias:.2f}%)
"""

if len(anomalias_detectadas) > 0:
    severidad_counts = anomalias_detectadas['severidad'].value_counts()
    resumen_deteccion += f"""

### Distribución por Severidad:
"""
    for severidad in ['CRÍTICO', 'ALERTA', 'ATENCIÓN', 'MENOR']:
        cantidad = severidad_counts.get(severidad, 0)
        porcentaje_sev = cantidad / len(anomalias_detectadas) * 100 if len(anomalias_detectadas) > 0 else 0
        resumen_deteccion += f"- **{severidad}:** {cantidad} anomalías ({porcentaje_sev:.1f}%)\n"
    
    # Análisis temporal
    dias_con_anomalias = len(anomalias_detectadas['fecha'].unique())
    dias_totales = (datos_agosto['timestamp'].max().date() - datos_agosto['timestamp'].min().date()).days + 1
    
    resumen_deteccion += f"""

### Análisis Temporal:
- **Días con anomalías:** {dias_con_anomalias} de {dias_totales} días totales
- **Cobertura temporal:** {dias_con_anomalias/dias_totales*100:.1f}% de los días
"""
    
    # Top 3 anomalías más críticas
    top_3_criticas = anomalias_detectadas.nsmallest(3, 'score_anomalia')
    resumen_deteccion += f"""

### Top 3 Anomalías Más Críticas:
"""
    for i, (idx, anomalia) in enumerate(top_3_criticas.iterrows(), 1):
        fecha = anomalia['timestamp'].strftime('%Y-%m-%d %H:%M')
        severidad = anomalia['severidad']
        thd = anomalia.get('THD', 0)
        score = anomalia['score_anomalia']
        resumen_deteccion += f"""
{i}. **{fecha}** - {severidad}
   - THD: {thd:.3f} | Score: {score:.3f}
   - {anomalia['diagnostico'][:100]}...
"""

else:
    resumen_deteccion += f"""

### Estado del Sistema:
✅ **NO SE DETECTARON ANOMALÍAS SIGNIFICATIVAS**
- El sistema indica operación normal durante todo agosto 2025
- Todos los parámetros dentro de rangos esperados
- THD promedio: {datos_agosto['THD'].mean():.3f} (dentro de límites normales)
"""

resumen_deteccion += f"""

---

## 🧠 DETALLES TÉCNICOS

### Modelo Utilizado:
- **Algoritmo:** Isolation Forest
- **Contaminación:** {modelo_cargado.contamination if hasattr(modelo_cargado, 'contamination') else 'N/A'}
- **Estimadores:** {modelo_cargado.n_estimators if hasattr(modelo_cargado, 'n_estimators') else 'N/A'}
- **Variables principales:** THD (Total Harmonic Distortion)

### Umbrales de Clasificación:
- **THD Crítico:** > {umbrales_thd['critico']}
- **THD Alerta:** > {umbrales_thd['alerta']}
- **THD Atención:** > {umbrales_thd['atencion']}
- **THD Normal:** ≤ {umbrales_thd['normal']}

### Estadísticas THD Agosto:
"""

if 'THD' in datos_agosto.columns:
    thd_stats = datos_agosto['THD'].describe()
    resumen_deteccion += f"""
- **Promedio:** {thd_stats['mean']:.3f}
- **Mediana:** {thd_stats['50%']:.3f}
- **Máximo:** {thd_stats['max']:.3f}
- **Mínimo:** {thd_stats['min']:.3f}
- **Desviación:** {thd_stats['std']:.3f}
- **Percentil 95:** {datos_agosto['THD'].quantile(0.95):.3f}
- **Percentil 99:** {datos_agosto['THD'].quantile(0.99):.3f}
"""

resumen_deteccion += f"""

---

## 💡 CONCLUSIONES Y RECOMENDACIONES

### Funcionamiento del Sistema:
✅ **Detección automática:** Sistema funcionando sin intervención humana
✅ **Análisis objetivo:** Decisiones basadas en patrones aprendidos
✅ **Clasificación inteligente:** Severidad asignada automáticamente
✅ **Monitoreo continuo:** Supervisión 24/7 de parámetros críticos

### Beneficios Identificados:
- **Detección temprana:** Identificación antes de fallas críticas
- **Objetividad:** Eliminación de sesgos humanos en detección
- **Consistencia:** Criterios uniformes de evaluación
- **Trazabilidad:** Registro completo de decisiones del modelo
- **Escalabilidad:** Capacidad de procesar grandes volúmenes de datos

### Próximos Pasos:
"""

if len(anomalias_detectadas) > 0:
    resumen_deteccion += f"""
1. **Generar OTs automáticas** para las {len(anomalias_detectadas)} anomalías detectadas
2. **Priorizar intervenciones** según clasificación de severidad
3. **Implementar monitoreo intensivo** para anomalías críticas
4. **Validar con eventos reales** para medir efectividad
5. **Ajustar umbrales** si es necesario basado en feedback
"""
else:
    resumen_deteccion += f"""
1. **Continuar monitoreo** automático para próximos períodos
2. **Mantener umbrales actuales** que muestran buen funcionamiento
3. **Documentar período estable** para referencia futura
4. **Preparar sistema** para detección en próximos meses
5. **Validar efectividad** cuando ocurran eventos reales
"""

resumen_deteccion += f"""

---

## 📁 ARCHIVOS GENERADOS

- `resultados_deteccion_completos.csv` - Todos los resultados de detección
- `anomalias_detectadas_agosto.csv` - Anomalías con clasificación detallada
- `deteccion_anomalias_agosto.png` - Visualización principal
- `analisis_detallado_anomalias.png` - Análisis estadístico

---

**Sistema de Detección Automática**
**Desarrollado por:** Antonio Cantos & Renzo Chavez
**TFM EADIC 2025** - Mantenimiento Predictivo con IA
**Fecha:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Guardar resumen
archivo_resumen = OUTPUT_PATH / 'resumen_deteccion_anomalias.md'
with open(archivo_resumen, 'w', encoding='utf-8') as f:
    f.write(resumen_deteccion)

print(f"📋 Resumen ejecutivo guardado: {archivo_resumen}")

# Mostrar resumen en pantalla
print(f"\n" + "="*70)
print(f"🎉 DETECCIÓN AUTOMÁTICA DE ANOMALÍAS - COMPLETADA")
print(f"="*70)
print(f"📊 Registros analizados: {len(resultados_deteccion):,}")
print(f"🔍 Anomalías detectadas: {total_anomalias:,} ({porcentaje_anomalias:.2f}%)")
print(f"⚙️ Método: Isolation Forest (automático)")
print(f"📁 Archivos generados: {len([f for f in OUTPUT_PATH.glob('*') if f.is_file()])} archivos")
print(f"⏰ Período: Agosto 2025 completo")
print(f"✅ Estado: Detección completada exitosamente")

if len(anomalias_detectadas) > 0:
    print(f"\n🚨 ANOMALÍAS POR SEVERIDAD:")
    for severidad in ['CRÍTICO', 'ALERTA', 'ATENCIÓN', 'MENOR']:
        cantidad = severidad_counts.get(severidad, 0)
        if cantidad > 0:
            print(f"   {severidad}: {cantidad} anomalías")
else:
    print(f"\n✅ RESULTADO: Operación normal - No se detectaron anomalías")

print(f"="*70)
print(f"\n📖 Consulta el resumen completo en: {archivo_resumen}")
print(f"📊 Revisa las visualizaciones en la carpeta: {OUTPUT_PATH}/")
print(f"🔧 Próximo paso: Generar OTs automáticas basadas en detecciones")